In [ ]:
class Strategy:
    def __init__(self, data) -> None:
        self.data = data
        self.indicator = None

data = [0,1,2,3,4,5]
strat = Strategy(data)
strat.__dict__.items()

In [ ]:
print(len.__name__)
print(len.__class__.__name__)

In [ ]:
import datetime as dt 
import yfinance as yf
import pandas as pd

num_days = 730
start=dt.date(2021, 1, 1)
start = max(start, dt.date.today() - dt.timedelta(days=num_days-1))
end = dt.date.today() 
print(start, end)
data = yf.download("SPY", interval="1h", start=start, end=end, prepost=True)
data = data.drop(['Adj Close'], axis=1)
data.index = list(map(lambda x: dt.datetime.strptime(str(x).replace(":",""), '%Y-%m-%d %H%M%S%z').replace(tzinfo=None), data.index))
data = data.rename_axis("Datetime").reset_index()
print(data)


In [ ]:
timestr = '2022-07-28 04:00:00'
output = dt.datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')
print(output)


In [ ]:
import bokeh.colors

# for c in dir(bokeh.colors.named):
    # print(c, c.__class__)
import bokeh.palettes
print(bokeh.palettes.Dark2_8)

# print(bokeh.colors.named.blue.__class__)
# print(all_colors)

# Top gainers aftermath

In [ ]:
from tradingview_screener import *
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (Query()
 .select('name', 'close', 'change', 'volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
 .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('change') > 5,
     Column('volume') > 1E6,
     Column('close') > 2,
    )
 .order_by('change', ascending=False)
 .get_scanner_data())

df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "change": "Change %",
    "float_shares_outstanding": "Float (M)"
    })
df = df[df.ticker.str.contains("NASDAQ:") | df.ticker.str.contains("NYSE:")]
print(len(df))
display(df)


In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

(dt.date(2024, 9, 1) - BDay(0)).date()

In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

# all_data = {dt.date.today() : df.ticker.to_list()}
fake_date_range = [dt.date(2024, 9, i) for i in range(1, 30)]
fake_data = {fake_date_range[i//2] : [str(j) for j in range(i, i+3)] for i in range(0, len(fake_date_range)*2,2)}
print(fake_data)


observation_time = 1 # days after making top gainer list
for today_date in fake_date_range:
    stocks_to_scan = set()
    for time_delta in range(observation_time + 1):
        # lookback_date = (today_date - BDay(time_delta)).date()
        lookback_date = today_date - dt.timedelta(days=time_delta)
        print(lookback_date)
        if lookback_date in fake_data:
            stocks_to_scan.update(fake_data[lookback_date])
            print("updated:", fake_data[lookback_date])
    print(today_date, stocks_to_scan)
    print()
        

# Premarket

In [ ]:
from tradingview_screener import *
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (
    Scanner.premarket_gainers
    .select('name', 'close', 'volume', 'premarket_change', 'premarket_volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
    .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('premarket_change') > 5,
     Column('premarket_volume') > 1E6,
     Column('close') > 1
    )
    .get_scanner_data())


df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.premarket_volume = df.premarket_volume * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "premarket_change": "Premarket %",
    "premarket_change_abs": "Premarket $",
    "premarket_volume": "Premarket Volume (M)",
    "float_shares_outstanding": "Float (M)"
    })
print(n_rows)

display(df)

In [ ]:
import datetime as dt
from download import *
from visualize import *

yf_df = download_from_yf("EBS", "1m", dt.date(2024,8,28), dt.date(2024,8,30))


In [ ]:
# pd.set_option('display.max_rows', 2500)
display(yf_df)

In [18]:
import datetime as dt
from download import *
from visualize import *

polygon_df = download_from_polygon("EBS", "4h", dt.date(2024,8,28), dt.date(2024,8,30))
display(polygon_df)

Polygon downloading:	EBS	4h	2024-08-28	2024-08-30


,Volume,Open,Close,High,Low,Datetime
0,1653.0,9.2800,9.2500,9.2900,9.2500,2024-08-28 04:00:00
1,1112238.0,9.2500,8.8400,9.5930,8.6200,2024-08-28 08:00:00
2,874286.0,8.8300,8.9100,8.9900,8.6800,2024-08-28 12:00:00
3,46097.0,8.9100,8.6200,8.9100,8.5100,2024-08-28 16:00:00
4,7878.0,8.8500,8.8900,9.0400,8.7800,2024-08-29 04:00:00
5,575879.0,8.8700,9.0800,9.4210,8.7903,2024-08-29 08:00:00
6,671643.0,9.0850,8.9100,9.1700,8.9100,2024-08-29 12:00:00
7,302915.0,8.9003,10.3000,10.7500,8.9003,2024-08-29 16:00:00
8,1424830.0,10.0000,10.2400,12.7700,10.0000,2024-08-30 04:00:00
9,5871954.0,10.6700,8.3888,11.8857,8.1500,2024-08-30 08:00:00


In [ ]:
pd.set_option('display.max_rows', 3000)
merged_df = yf_df.merge(polygon_df, how='left')
display(merged_df)
pd.set_option('display.max_rows', 10)

In [ ]:
market_open = pd.to_datetime('09:30:00').time()
market_close = pd.to_datetime('16:00:00').time()

pre_post_data = polygon_df.loc[(polygon_df['Datetime'].dt.time <= market_open) | (polygon_df['Datetime'].dt.time >= market_close)]

# TODO: Lookup not by exact value, but by the closest timestamp in yf_df after the query timestamp. 
for index, row in pre_post_data.iterrows():
    yf_row = yf_df[yf_df['Datetime'] == row.Datetime]
    assert len(yf_row) <= 1, "Multiple of the same timestamps found."
    yf_df.loc[yf_row.index, 'Volume'] = row.Volume
    

# pre_post_data["closest_timestamp"] = pd.merge_asof(pre_post_data, yf_df, on='Datetime', direction='forward')['Datetime']

In [ ]:
import pandas_market_calendars as mcal
import pandas as pd
import datetime as dt

market_schedule = mcal.get_calendar('NYSE').schedule(start_date='2024-01-01', end_date='2024-12-31')
regular_business_days = market_schedule.loc[market_schedule['market_close'].dt.time >= pd.to_datetime('20:00:00').time()].index.to_list()

regular_business_days = [i.to_pydatetime().date() for i in regular_business_days]
print(regular_business_days)
print((dt.datetime.now().date() + dt.timedelta(days=1)) in regular_business_days)

In [ ]:
import yfinance as yf
import datetime as dt
import requests
import pandas as pd

interval = 'Daily' # '5min'
# url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval={interval}&outputsize=full&apikey=demo"
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey=demo"
r = requests.get(url)
data = r.json()

# data = yf.download('IBM', interval='1d', start=dt.datetime(1914,1,1), end=dt.datetime.today(), prepost=True, progress=False)
# display(data)

df = pd.DataFrame(columns=["Datetime", 'Open', 'High', 'Low', 'Close', 'Volume'])
for timestamp, ohlcv in data[f"Time Series ({interval})"].items():
    df = pd.concat([pd.DataFrame([[dt.datetime.fromisoformat(timestamp),
                     ohlcv['1. open'],
                     ohlcv['2. high'],
                     ohlcv['3. low'],
                     ohlcv['4. close'],
                     ohlcv['5. volume']]], columns=df.columns), df], ignore_index=True)
display(df)

In [ ]:
from download import * 

df = get_historical_data("EBS", "1m", save_to_pickle=False)

In [ ]:
pd.set_option('display.max_rows', 2500)
display(df)
pd.set_option('display.max_rows', 100)

In [ ]:
from bisect import bisect_left
import datetime as dt
from download import * 
for query_date in [dt.date(2024, 1, 1) + dt.timedelta(days=i) for i in range(365)]:
    result = most_recent_business_day(query_date)
    print(query_date, result)

In [ ]:
'''
References: 
https://www.projectpro.io/recipes/upload-files-to-google-drive-using-python
https://stackoverflow.com/questions/75454425/access-blocked-project-has-not-completed-the-google-verification-process
https://pythonhosted.org/PyDrive/oauth.html
'''
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

file = 'C:\\Users\\georg\\GitHub\\money-tree\\test_run.html'

gauth = GoogleAuth(settings_file='C:\\Users\\georg\\GitHub\\money-tree\\settings.yaml', http_timeout=30)
drive = GoogleDrive(gauth)
upload_file_list = [file] 
for upload_file in upload_file_list: 
    gfile = drive.CreateFile({'parents': [{'id': '1N_7YUWKB2ezW_d_fse719Xo_toFuqx0q'}]}) # Read file and set it as the content of this instance. 
    gfile.SetContentFile(upload_file) 
    gfile.Upload() # Upload the file.

# list file
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format('1N_7YUWKB2ezW_d_fse719Xo_toFuqx0q')}).GetList() 
for file in file_list: 
    print('title: %s, id: %s' % (file['title'], file['id']))

# # download file
# for i, file in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
#     print('Downloading {} file from GDrive ({}/{})'.format(file['title'], i, len(file_list))) 
#     file.GetContentFile(file['title'])

In [ ]:
import sys
print(sys.platform)
import platform
print(platform.system())


In [ ]:
from download import * 
import os
test_pickle_path = pickle_filepath("NVDA", "1d")
print(test_pickle_path)
print(os.path.exists(test_pickle_path))


In [5]:
import yfinance as yf
import datetime as dt
from download import * 

data = download_from_polygon("NVDA", "1h", start=dt.date(2024, 9, 10), end = dt.date(2024, 9, 10))
display(data)

data = download_from_yf("NVDA", "1h", start=dt.date(2024, 9, 10), end = dt.date(2024, 9, 10))
display(data)


,Volume,Open,Close,High,Low,Datetime
0,419832.0,106.8700,106.1400,106.87,106.0500,2024-09-10 04:00:00
1,382139.0,106.1200,106.1500,106.30,105.5800,2024-09-10 05:00:00
2,195911.0,106.1900,106.3500,106.49,105.9500,2024-09-10 06:00:00
3,1149976.0,106.3500,107.0200,107.20,106.2500,2024-09-10 07:00:00
4,3263159.0,106.3900,107.8800,107.95,105.7047,2024-09-10 08:00:00
5,51728991.0,107.8800,106.9919,109.40,106.5300,2024-09-10 09:00:00
6,49711043.0,106.9900,106.8701,107.80,105.9600,2024-09-10 10:00:00
7,33803926.0,106.8700,105.2800,107.44,105.2500,2024-09-10 11:00:00
8,29752958.0,105.2850,105.8109,106.25,104.9500,2024-09-10 12:00:00
9,21229582.0,105.8101,106.5450,106.72,105.7600,2024-09-10 13:00:00


,Datetime,Open,High,Low,Close,Volume
0,2024-09-10 04:00:00,106.760000,106.830000,106.230000,106.260000,0
1,2024-09-10 05:00:00,106.300000,106.400000,105.580000,106.040000,0
2,2024-09-10 06:00:00,106.050000,106.310000,105.910000,106.160000,0
3,2024-09-10 07:00:00,106.190000,106.900000,106.180000,106.870000,0
4,2024-09-10 08:00:00,106.860000,107.840000,105.596400,107.780000,0
5,2024-09-10 09:00:00,107.770000,108.390000,104.790000,107.810000,0
6,2024-09-10 09:30:00,108.044998,109.370003,105.959999,107.290001,87845460
7,2024-09-10 10:30:00,107.240501,107.544998,106.416000,107.117798,33241851
8,2024-09-10 11:30:00,107.099998,107.440002,104.949997,105.455002,36994246
9,2024-09-10 12:30:00,105.449997,106.650002,105.410004,106.199997,24532466


In [ ]:
from download import * 
data = get_historical_data("NVDA", "1h")
display(data)

In [4]:
import pickle
import os
import pandas as pd

filepath = "G:\\My Drive\\data\\QH_1h.pkl"
print(os.path.exists(filepath))
with open(filepath, 'rb') as f:
    data = pickle.load(f)
pd.set_option('display.max_rows', 2500)
display(data)
pd.set_option('display.max_rows', 100)



True


,Datetime,Open,High,Low,Close,Volume
0,2024-09-10 06:00:00,0.370000,0.5200,0.3700,0.42930,1628319.0
1,2024-09-10 07:00:00,0.430000,0.4329,0.3100,0.33190,1179547.0
2,2024-09-10 08:00:00,0.405700,0.8160,0.3102,0.53800,12009639.0
3,2024-09-10 09:00:00,0.543700,0.6177,0.4337,0.50000,23289564.0
4,2024-09-10 10:00:00,0.501183,0.7700,0.4726,0.71000,63346078.0
5,2024-09-10 11:00:00,0.710031,0.8504,0.6336,0.74335,49794648.0
6,2024-09-10 12:00:00,0.744000,0.9900,0.7000,0.81620,54086289.0
7,2024-09-10 13:00:00,0.817000,1.0400,0.8000,0.94020,45049491.0
8,2024-09-10 13:00:00,0.817000,1.0400,0.8000,0.94020,45054491.0
9,2024-09-10 14:00:00,0.947500,0.9475,0.7722,0.80480,19063570.0


In [3]:
from download import *

is_continuous(data['Datetime'], '4h')

Business days loaded!


c:\Users\georg\GitHub\money-tree\download.py:78: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  day_times = pd.date_range(start=day_start, end=day_end, freq=INTERVALS_TO_PD_OFFSET[interval])


(False,
 array(['2024-09-09T04:00:00.000000000', '2024-09-10T04:00:00.000000000'],
       dtype='datetime64[ns]'))